## TCP for Reliable Transport in IP Networks
The IP protocol provides a mechanism to send data (in the form of IP packets) from a source to a destination via the network. However, the IP protocol does not provide any guarantees on the delivery of the packet. It undertakes transmission and forwarding of packets on a best efforts basis.

 However, most applications need to know if the data has been received at the receiver. For example, in a file transfer application, the source needs to know which all parts of the file have been received so as to either send the remaining part of the file or to stop transmission once all the data is received. This demands the need for a reliable data transport mechanism. This data reliability is provided by the TCP protocol. TCP achieves this by using acknowledgement (ACK) packets from the destination.

 An ACK packet in TCP contains a sequence number which indicates the next byte in the data byte-stream expected at the receiver (indicating that all bytes upto the sequence number have been received succesfully). The receiver could have received packets beyond what the sequence number indicates, but without the current sequence number byte, it cannot go forward in the reconstruction of the byte-stream.

 ### Sliding Window protocol.

  The sliding window protocol can establish a rudimentary reliable transport mechanism between the transmitter and receiver. Let $B_{Tx}$ denote the packet buffer length at the transmitter. $B_{Tx}$ corresponds to the maximum number of bytes transmitted by the sender, but yet to be acknowledged. When an ACK is received, all bytes upto the sequence number mentioned in the ACK packet is removed from the buffer and the same number of bytes are pushed into the buffer and transmitted (assuming that there is data to be transmitted). Thus, at any given time upto $B_{Tx}$ bytes are in the network. 

  Let us now do a quick analysis of the efficiency of the sliding window protocol. Assume a loss-less link between the transmitter and the receiver, i.e., none of the packets are lost in transmission. Let $C$ (bytes/second) denote the capacity of the link  between the transmitter and the receiver (in a multi-hop link, this will be link with the least capacity). Let $RTT$ (seconds) denote the round trip time between the transmitter and the receiver. Let $w$ denote the size of a packet. Then, once a packet is transmitted, an ack is received $RTT + \frac{w}{C}$ seconds later. Assuming $B_{Tx}$ bytes were transmitted initially, the acknowledgemnt for the last byte will be received at $RTT + \frac{B_{Tx}}{C}$. Similarly the acknowledgement for the next batch of $B_{Tx}$ bytes will be received at $RTT + \frac{B_{Tx}}{C} + RTT + \frac{w}{C}$ and so on. Hence, after k rounds of such transmissions, $k B_{Tx}$ bytes are transmitted in $k RTT + \frac{B_{Tx}}{C} + k \frac{w}{C}$ seconds. Thus the effective throughput seen at the transmitter would be 
  \begin{align}
  C' &= \lim_{k \rightarrow \infty} \frac{k B_{Tx}}{k RTT + \frac{B_{Tx}}{C} + k \frac{w}{C}}\\
 &= \frac{B_{Tx}}{RTT + \frac{w}{C}}\\
&\sim \frac{B_{Tx}}{RTT},
  \end{align}
where in the last step we assumed that $C \times RTT >> w$. Thus the effective capacity monotonically increases as $B_{Tx}$ increases and reaches the maximum possible $C$, when $B_{Tx} = C RTT$. Increasing $B_{Tx}$ beyond $C RTT$ does not increase the effective capacity, but only helps to queue up more bytes in the transmit buffer. At any instance of time, the number of bytes not transmitted by waiting in the buffer is $B_{Tx} - C \times RTT$. Hence, for a new byte entering the buffer, the waiting time in the queue would be $\frac{B_{Tx} - C \times RTT}{C}$.

### Optimal buffer size
From the above discussion, we notice that the sweet spot at which the throughput is maximised and the queueing delay is minimised is at $B_{Tx} = C \times RTT$, also known as the bandwidth delay product (BDP).

### Packet loses and re-transmission logic
In the above discussion, we assumed that none of the packets would be lost in transit. However, that is not the case in reality. If we continue using the above logic in a lossy network, the TCP flow will freeze the moment a packet is lost. To circumvent this scenario, one could use a re-transmission logic, which for example could wait for a prescribed time for an ACK packet and in the absence of an ACK packet, the packet can be re-transmitted. Choosing the appropriate re-transmission timer threshold then becomes crucial. A timer threshold smaller than the actual RTT would cause un-necessary re-transmissions. A re-transmission timer larger than RTT will result in queueing delays at the transmitter.

### Dynamic nature of channel capacity and RTT
In the above section, we saw that for a fixed channel capacity and round trip time (RTT), buffer length equal to the bandwidth delay product was an optimal choice. However, in actual systems, both the channel capacity and the round trip time would vary with time. In such a scenario, the question is how should one adjust the buffer length and the re-transmission timers?

##Congestion Collapse
 Congestion collapse is the phenomenon where increase in the number of flows in a link results in reduction in the effective utility of the link. That is, as congestion and demand increases in a link, the link performance deteriorates. This was first observed in 1984 by John Nagle [reference] while internetworking Ford network and ARPANET and in 1986 by Van Jacobson [reference] while interconnecting two networs in Berkeley. 

 ### Simple example for congestion collapse

 Consider a lossless network with infinite buffers at all intermediate nodes. Assume that for each TCP flow, the buffer size $B_{Tx}$ is fixed. Let $RTT$ be the common round trip time for all flows. Let $C$ be the total capacity of the link. Let $n$ denote the number of simultaneous flows present, thus making available $\frac{C}{n}$ capacity for each flow. 
 
Let the fraction $u = \frac{throughput}{latency}$ denote the utility of the network. From our discussion above on sliding window protocol we know that as long as $ B_{Tx} < \frac{C}{n} RTT$, queueing delay will be zero for each of the flows, and hence the total utility in the network would be $$u = n \frac{B_{Tx}}{RTT} \frac{1}{RTT}.$$ However, when $B_{Tx} > \frac{C}{n}RTT$, the capacity for each link is capped at $\frac{C}{n}$, and the end to end delay increases to $RTT + (B_{Tx} - \frac{C}{n}RTT)\frac{n}{C}$ = $\frac{nB_{Tx}}{C}$. Thus, the utility now becomes $$u = \frac{C^2}{nB_{Tx}},$$ a decreasing function in $n$.

#### Exercise: 

For a fixed network capacity $C$ and fixed buffer length $B_{Tx}$, plot utility $u$ against number of flows $n$.

 In the above example, we considered a lossless network with no re-transmissions. Now consider a network where a packet is re-transmitted if an ACK is not received in $T_{RT}$ time units. We observe that the moment, the end-to-end delay exceeds $\frac{nB_{Tx}}{C}$, a re-transmission packet is queued for transmission, thereby getting into a self loop of repeatedly sending the same packet and thereby achieving a throughput of zero.

## Congestion Control

In this section, we will describe the basic form of the TCP congestion control algorithm.

Congestion control algorithms adpatively adusts the transmit buffer size and the round trip time estimate, so that the transmit buffer size is close to the available bandwidth delay product. The buffer size adaptation follows an additive increase multiplicative decrease (AIMD) policy. The algorithm has two phases:

1) Slow start
  * In slow start phase, for every packet ACKed, the transmit buffer size is incremented by the size of the packet (additive increase). This phase continues till the buffer size crosses the dynamic slow start threshold (*ssthresh*), beyond which the congestion avoidance phase takes over.

2) Congestion avoidance
  * In the congestion avoidance phase, the buffer is incremented by a packet size every round trip time (RTT) estimate.

**Round trip timer**: The sender also keeps estimates of the round trip time (smoothened RTT, $SRTT$), and the variance of the round trip time ($RTT_{VAR}$). Further, based on these two estimates the roundtrip timeout (RTO) is estimated as $RTO = SRTT + 4 * RTT_{VAR}$. On reception of an ACK packet with round trip delay $R'$, $SRTT$ and $RTT_{VAR}$ estimates are updated as follows:
\begin{align}
RTT_{VAR} &=  (1-\beta) RTT_{VAR} + \beta \vert SRTT - R' \vert\\
SRTT &= (1-\alpha) SRTT + \alpha R'\\
RTO &= SRTT + 4*RTT_{VAR},
\end{align}
where the recommended values for $\alpha$ and $\beta$ are $1/8$ and $1/4$, respectively.


**Packet Loss**:
How is a packet loss defined? A packet is defined to be lost if it's corresponding ACK is not received within the timeout ($RTO$) time. When a packet loss is detected, the algorithm reduces *ssthresh* as follows:
$ssthresh = max(FlightSize/2, 2 * SMSS)$, where $FlightSize$ denotes the number of packets in the network (unacknowledged packets). Further, the transmit buffer is reduced to the size of 1 packet.

##Active Queue Management (AQM)
In the previous sections we saw how TCP handles congestion. So far in the examples we saw, the only entity that was actively involved in congestion control was the sender. The sender uses the signalling implicit in packet drops to control congestion. The network elements were not required to participate in congestion control mechanisms. The question then is, can network elements contribute to congestion control mechanisms, and thereby also improve the efficiency of the network? Active queue management in the network elements augments the TCP congestion control algorithms and tries to operate the network efficiently. Recommendations by IETF on use of AQM for networks elements is given in [RFC 7567](https://tools.ietf.org/html/rfc7567).

An ideal operating point for a network is to have maximum utilisation of the network and minimum queueing delays in the network. However, these are conflicting requirements, as an increase in network utilisation results in increased queueing delays. 

As far as a network element is considered it's goals could be listed as:
* maximise network throughput,
* minimise queueing delays,
* ensure fair allocation of resources among flows.

To achieve these goals, one mechanism the network elements can use is delebrate packet drops. Packet drops can act as a means to limit the buffer sizes in the network elements and also as a means to signal network state to the end points.

## Tail drop
Consider a network element with a finite buffer size. In tail drop scheme, the network element drops a packet if the buffer is full. Tail drop is a simple queue management scheme, but has the following drawbacks:
* As the network load approaches capacity, the buffer sizes also approach full buffer length. Thus, the element will introduce a queueing delay corresponding to it's buffer size.
* Bursty traffic is adversely affected as there is a higher chance of dropping packets from a bursty flow.
* Flow synchronisation: When the buffer size is full, multiple flows will sense packet drops at similar times. This can potentially result in a synchronisation in the congestion control procedures of the different flow, potentially resulting in an oscillatory load on the network.

## Random Early Drop (RED)
 Random Early Drop (RED) is a probabilistic packet drop mechanism that tries to address some of the drawbacks of Tail drop. RED mechanism is described below. On the arrival of a new packet, the following decisions are made:
 * If the queue length is smaller than a lower threshold $(Q_{min})$, the new packet is admitted into the queue.
 * If the queue length is larger than a larger threshold $Q_{max}$, the new packet is discarded. 
 * If the queue length is larger than $Q_{min}$ but smaller than $Q_{max}$, then the packet is admitted with probability $p$, $( 0 <= p <=1)$.

 RED has the following properties:
 * Since the packets are randomly dropped, it prevents synchronsation of packet drops.
 * Since packets are dropped earlier than when the queue is full, it signals congestion earlier than Tail Drop.
 * It can keep the average queue length in control.
 * Works well for responsive flows (TCP flows). In the presence of unresponsive flows (UDP), the responsive flows can get suppressed.


 ## $m$-CHOKe

  The two algorithms (Tail drop, RED) discussed above, do not differentiate between flows, and hence they are not effective in selectively suppressing un-responsive flows (UDP flows). Consequently, unresponsive flows can game the system and get an upper hand.
[Add plot of UDP vs TCP throughput in congestion.]
  
  CHOKe (Choose to Keep responsive flows, Choose to Kill unresponsive flows) tries to identify and penalise unresponsive flows. In its simplest form, like RED, it keeps two thresholds $Q_{min}$ and $Q_{max}$, such that whenever the queue length is less than $Q_{min}$, all new packets are admitted into the queue, and whenever the queue lenngth is larger than $Q_{max}$, all new packets are dropped. In betweeen these thresholds, on arrival of a new packet, the algorithm does the following:
  * Pick $m$ packets at random from the queue.
    * If the flow id of the incoming packet and any of the selected packets match, then drop all these packets.
    * Else, admit the new packet into the queue.

Such a mechanism disicentivises an unresponsive flow that keeps pumping more packets into the system.

[Add a comparative plot on the behavior of different AQM techniques].

### Exercises
[TODO]